1. Tirar as linhas que tem pelo menos 4 valores nulos ou maior-> ignorar essas instancias
2. Substituir tbd por -1 -> completar automaticamente
3. Isso é limpeza dos dados
4. Transformar cada classe em uma coluna
5. Separar jogos que são multi plataformas, correlação entre as avaliações

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

Uma vez que temos dois arquivos de csv, iremos juntar os dois, utilizando o concat.

In [3]:
dados1 = pd.read_csv( "trabalho2_dados_21.csv")
dados2 = pd.read_csv("trabalho3_dados_adicionais_21.csv")
df = pd.concat([dados1,dados2], ignore_index=True)

In [8]:
dados1.head()

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Green Lantern: Rise of the Manhunters,X360,Action,Warner Bros. Interactive Entertainment,0.10,7-Jun-11,59.0,17.0,7,22.0,Double Helix Games
1,Kinect Sports: Season Two,X360,Sports,Microsoft Game Studios,2.18,25-Oct-11,66.0,46.0,6.3,27.0,"BigPark, Rare Ltd."
2,Brothers in Arms: Double Time,Wii,Shooter,Ubisoft,0.12,23-Sep-08,45.0,13.0,6.6,17.0,Gearbox Software
3,Steambot Chronicles: Battle Tournament,PSP,Fighting,Irem Software Engineering,0.06,30-Jun-09,54.0,14.0,6.1,7.0,Irem
4,Tiger Woods PGA Tour 07,Wii,Sports,Electronic Arts,0.46,13-Mar-07,71.0,41.0,6.9,44.0,EA Redwood Shores


In [9]:
dados2.head()

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Tiger Woods PGA Tour 06,X360,Sports,Electronic Arts,0.30,16-Nov-05,71.0,36.0,7.4,8.0,EA Sports
1,Peter Jackson's King Kong: The Official Game o...,X360,Action,Ubisoft,0.23,21-Nov-05,80.0,57.0,7.2,60.0,Ubisoft Montpellier
2,Pinball Hall of Fame: The Gottlieb Collection,PSP,Misc,Play It,0.22,6-Dec-05,73.0,27.0,9.5,11.0,Farsight Studios
3,Tiger Woods PGA Tour 06,PSP,Sports,Electronic Arts,0.20,26-Sep-05,80.0,15.0,9,7.0,Team Fusion
4,Peter Jackson's King Kong: The Official Game o...,PSP,Action,Ubisoft,0.18,20-Dec-05,56.0,10.0,6.2,20.0,Phoenix Games Studio


Vamos analisar os tipo de cada variável no dataframe.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3196 entries, 0 to 3195
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   nome                3196 non-null   object 
 1   plataforma          3196 non-null   object 
 2   genero              3196 non-null   object 
 3   editora             3192 non-null   object 
 4   vendas              3196 non-null   float64
 5   lancamento          2490 non-null   object 
 6   avaliacao-criticos  2050 non-null   float64
 7   numero-criticos     2050 non-null   float64
 8   avaliacao-usuarios  2446 non-null   object 
 9   numero-usuarios     1944 non-null   float64
 10  fabricante          2473 non-null   object 
dtypes: float64(4), object(7)
memory usage: 274.8+ KB


Observer que a quantidade de valores em avaliacao-usuarios é maior que o numero de usuarios. Além disso, não possuímos as mesmas qauntidades em todas as variáveis. Dependendo do modelo a ser escolhido, essa diferença de tamanho pode representar uma dificuldade.

In [6]:
df.head()

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Green Lantern: Rise of the Manhunters,X360,Action,Warner Bros. Interactive Entertainment,0.10,7-Jun-11,59.0,17.0,7,22.0,Double Helix Games
1,Kinect Sports: Season Two,X360,Sports,Microsoft Game Studios,2.18,25-Oct-11,66.0,46.0,6.3,27.0,"BigPark, Rare Ltd."
2,Brothers in Arms: Double Time,Wii,Shooter,Ubisoft,0.12,23-Sep-08,45.0,13.0,6.6,17.0,Gearbox Software
3,Steambot Chronicles: Battle Tournament,PSP,Fighting,Irem Software Engineering,0.06,30-Jun-09,54.0,14.0,6.1,7.0,Irem
4,Tiger Woods PGA Tour 07,Wii,Sports,Electronic Arts,0.46,13-Mar-07,71.0,41.0,6.9,44.0,EA Redwood Shores


Há algumas variáveis que seria interessante transformá-las, para que possam ser utilizadas em um modelo. São elas: plataforma, pois nós possuímos 9 classes e seria interessante termos a informação dos tipos de plataformas que um jogo foi produzido em uma única instância; genero seria interessante transformar em numérico; avaliacao-usuarios deve ser um valor numérico;

In [7]:
df['plataforma'].unique()

array(['X360', 'Wii', 'PSP', 'PS3', 'PS4', '3DS', 'PSV', 'XOne', 'WiiU'],
      dtype=object)

Antes de passarmos para essas etapas citadas, iremos limpar os dados.

Como vamos começar a retirar os valores nulos, irei passar a coluna de avaliação dos usuários para float e substituir 'tbd' por nan.

In [4]:
df['avaliacao-usuarios'] = pd.to_numeric(df['avaliacao-usuarios'].replace('tbd',np.nan),errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3196 entries, 0 to 3195
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   nome                3196 non-null   object 
 1   plataforma          3196 non-null   object 
 2   genero              3196 non-null   object 
 3   editora             3192 non-null   object 
 4   vendas              3196 non-null   float64
 5   lancamento          2490 non-null   object 
 6   avaliacao-criticos  2050 non-null   float64
 7   numero-criticos     2050 non-null   float64
 8   avaliacao-usuarios  1944 non-null   float64
 9   numero-usuarios     1944 non-null   float64
 10  fabricante          2473 non-null   object 
dtypes: float64(5), object(6)
memory usage: 274.8+ KB


In [5]:
df

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Green Lantern: Rise of the Manhunters,X360,Action,Warner Bros. Interactive Entertainment,0.10,7-Jun-11,59.0,17.0,7.0,22.0,Double Helix Games
1,Kinect Sports: Season Two,X360,Sports,Microsoft Game Studios,2.18,25-Oct-11,66.0,46.0,6.3,27.0,"BigPark, Rare Ltd."
2,Brothers in Arms: Double Time,Wii,Shooter,Ubisoft,0.12,23-Sep-08,45.0,13.0,6.6,17.0,Gearbox Software
3,Steambot Chronicles: Battle Tournament,PSP,Fighting,Irem Software Engineering,0.06,30-Jun-09,54.0,14.0,6.1,7.0,Irem
4,Tiger Woods PGA Tour 07,Wii,Sports,Electronic Arts,0.46,13-Mar-07,71.0,41.0,6.9,44.0,EA Redwood Shores
...,...,...,...,...,...,...,...,...,...,...,...
3191,Port Royale 3,X360,Simulation,Kalypso Media,0.04,9-Oct-12,56.0,11.0,4.5,6.0,Kalypso
3192,Major League Baseball 2K8,PSP,Sports,Unknown,0.03,3-Mar-08,63.0,5.0,NaN,NaN,Kush Games
3193,Monster Hunter Frontier Online,PS3,Role-Playing,NaN,0.03,NaN,NaN,NaN,NaN,NaN,NaN
3194,Battle vs. Chess,PS3,Misc,TopWare Interactive,0.03,TBA,NaN,NaN,NaN,NaN,TopWare Interactive


Ao olhar os arquivos csv, pudemos observar que as instâncias que possuem mais de 3 campos vazios, colunas, podem ser retirados do dataframe, pois estas instâncias não possuem valores de avaliação. 

In [9]:
display(df)

,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Green Lantern: Rise of the Manhunters,X360,Action,Warner Bros. Interactive Entertainment,0.10,7-Jun-11,59.0,17.0,7.0,22.0,Double Helix Games
1,Kinect Sports: Season Two,X360,Sports,Microsoft Game Studios,2.18,25-Oct-11,66.0,46.0,6.3,27.0,"BigPark, Rare Ltd."
2,Brothers in Arms: Double Time,Wii,Shooter,Ubisoft,0.12,23-Sep-08,45.0,13.0,6.6,17.0,Gearbox Software
3,Steambot Chronicles: Battle Tournament,PSP,Fighting,Irem Software Engineering,0.06,30-Jun-09,54.0,14.0,6.1,7.0,Irem
4,Tiger Woods PGA Tour 07,Wii,Sports,Electronic Arts,0.46,13-Mar-07,71.0,41.0,6.9,44.0,EA Redwood Shores
...,...,...,...,...,...,...,...,...,...,...,...
1559,Port Royale 3,X360,Simulation,Kalypso Media,0.04,9-Oct-12,56.0,11.0,4.5,6.0,Kalypso
1560,Major League Baseball 2K8,PSP,Sports,Unknown,0.03,3-Mar-08,63.0,5.0,-1.0,NaN,Kush Games
1561,Monster Hunter Frontier Online,PS3,Role-Playing,NaN,0.03,NaN,NaN,NaN,NaN,NaN,NaN
1562,Battle vs. Chess,PS3,Misc,TopWare Interactive,0.03,TBA,NaN,NaN,NaN,NaN,TopWare Interactive


In [6]:
## Retirando as instâncias que possuem mais de 3 campos sem preencher
indice = lambda x: df.iloc[x].isnull().sum()
guarda_index=[]
for i in df.index:
    if indice(i)>=3:
        guarda_index.append(i)
df.drop(guarda_index, inplace=True)
#renomear os indices
indice_novo = [i for i in range(len(df))]
for i in range(len(df)):
    df.rename({df.index[i]:indice_novo[i]}, inplace=True,axis='index')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2164 entries, 0 to 2163
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   nome                2164 non-null   object 
 1   plataforma          2164 non-null   object 
 2   genero              2164 non-null   object 
 3   editora             2164 non-null   object 
 4   vendas              2164 non-null   float64
 5   lancamento          2164 non-null   object 
 6   avaliacao-criticos  2050 non-null   float64
 7   numero-criticos     2050 non-null   float64
 8   avaliacao-usuarios  1944 non-null   float64
 9   numero-usuarios     1944 non-null   float64
 10  fabricante          2156 non-null   object 
dtypes: float64(5), object(6)
memory usage: 202.9+ KB


Agora, iremos transformar os dados das colunas plataforma, separando cada classe em uma coluna do tipo booleana. Por consequência, irá aparecer duas colunas de avaliação, usuário e crítico, e duas de quantidade, de usuários e críticos, para cada classe de plataforma.

In [127]:
#retirando a coluna de datas e as linhas duplicadas
df.drop(columns='lancamento',inplace=True)
df = df.drop_duplicates()

In [1]:
# [i for i in df.groupby(['nome'])['vendas'].sum()]

In [9]:
df_novo = pd.DataFrame(columns=['nome','genero','editora','vendas', 'X360', 'Wii', 'PSP', 'PS3', 
                                'PS4', '3DS', 'PSV', 'XOne', 'WiiU',
                                'avaliacao_criticos_novo','numero_criticos_novo',
                                'avaliacao_usuarios_novo','numero_usuarios_novo','fabricante'], 
                       index=[i for i in range(len(df.groupby(['nome'])))])

In [136]:
df.keys()

Index(['nome', 'plataforma', 'genero', 'editora', 'vendas',
       'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios',
       'numero-usuarios', 'fabricante'],
      dtype='object')

In [138]:
#iremos agrupar a análise pelo nome dos jogos
gb = df.groupby(['nome'])
#preenchendo as colunas que não precisaram ser transformadas
df_novo['nome'] = [i[0] for i in gb['nome']]
df_novo['vendas'] = [i for i in gb['vendas'].sum()]
df_novo['editora'] = [i[1].iloc[0] for i in gb['editora']]
df_novo['fabricante'] = [i[1].iloc[0] for i in gb['fabricante']]
df_novo['numero_criticos_novo'] = [i for i in gb['numero-criticos'].sum()]
df_novo['numero_usuarios_novo'] = [i for i in gb['numero-usuarios'].sum()]

In [56]:
#preenchendo genero
dic_genero = {'Action':1,'Sports':2, 'Shooter':3, 'Fighting':4, 'Role-Playing':5, 'Misc':6,
       'Strategy':7, 'Adventure':8, 'Racing':9, 'Platform':10, 'Simulation':11,
       'Puzzle':12}
def genero(c):
    var=[i[1].iloc[0] for i in gb['genero']]
    numero_corr = [dic_genero[i] for i in var]
    return numero_corr


df_novo['genero'] = gb.apply(genero)[0]

In [124]:
#preeenchendo as colunas referente as plataformas, versao teste
def x360(c):
    var=[i[1] for i in gb['plataforma']]
    plataforma_corr = []
    for i in var:
        if len(i.loc[i.iloc[:]=='X360'])>0:
            plataforma_corr.append(1)
        else:
            plataforma_corr.append(0)

    return plataforma_corr
df_novo['X360'] = gb.apply(x360)[0]


def wii(c):
    var=[i[1] for i in gb['plataforma']]
    plataforma_corr = []
    for i in var:
        if len(i.loc[i.iloc[:]=='Wii'])>0:
            plataforma_corr.append(1)
        else:
            plataforma_corr.append(0)
    return plataforma_corr
df_novo['Wii'] = gb.apply(wii)[0]

def psp(c):
    var=[i[1] for i in gb['plataforma']]
    plataforma_corr = []
    for i in var:
        if len(i.loc[i.iloc[:]=='PSP'])>0:
            plataforma_corr.append(1)
        else:
            plataforma_corr.append(0)
    return plataforma_corr
df_novo['PSP'] = gb.apply(psp)[0]

def ps3(c):
    var=[i[1] for i in gb['plataforma']]
    plataforma_corr = []
    for i in var:
        if len(i.loc[i.iloc[:]=='PS3'])>0:
            plataforma_corr.append(1)
        else:
            plataforma_corr.append(0)
    return plataforma_corr
df_novo['PS3'] = gb.apply(ps3)[0]


def ps4(c):
    var=[i[1] for i in gb['plataforma']]
    plataforma_corr = []
    for i in var:
        if len(i.loc[i.iloc[:]=='PS4'])>0:
            plataforma_corr.append(1)
        else:
            plataforma_corr.append(0)
    return plataforma_corr
df_novo['PS4'] = gb.apply(ps4)[0]

def ds3(c):
    var=[i[1] for i in gb['plataforma']]
    plataforma_corr = []
    for i in var:
        if len(i.loc[i.iloc[:]=='3DS'])>0:
            plataforma_corr.append(1)
        else:
            plataforma_corr.append(0)
    return plataforma_corr
df_novo['3DS'] = gb.apply(ds3)[0]


def psv(c):
    var=[i[1] for i in gb['plataforma']]
    plataforma_corr = []
    for i in var:
        if len(i.loc[i.iloc[:]=='PSV'])>0:
            plataforma_corr.append(1)
        else:
            plataforma_corr.append(0)
    return plataforma_corr
df_novo['PSV'] = gb.apply(psv)[0]

def xone(c):
    var=[i[1] for i in gb['plataforma']]
    plataforma_corr = []
    for i in var:
        if len(i.loc[i.iloc[:]=='XOne'])>0:
            plataforma_corr.append(1)
        else:
            plataforma_corr.append(0)
    return plataforma_corr
df_novo['XOne'] = gb.apply(xone)[0]


def wiiu(c):
    var=[i[1] for i in gb['plataforma']]
    plataforma_corr = []
    for i in var:
        if len(i.loc[i.iloc[:]=='WiiU'])>0:
            plataforma_corr.append(1)
        else:
            plataforma_corr.append(0)
    return plataforma_corr
df_novo['WiiU'] = gb.apply(wiiu)[0]

In [140]:
#preencher as avaliações dos usuários e dos críticos

def aval_critic(c):
    var=[i[1].loc[:,['avaliacao-criticos', 'numero-criticos']] for i in gb['avaliacao-criticos', 'numero-criticos']]
    aval_corr = []
    for i in var:
        peso_aval_soma=0
        for tab in range(len(i)):
            aux = i.iloc[tab]
            peso_aval_soma += aux[0]*aux[1]
        aval_corr.append((peso_aval_soma)/(i.iloc[:,1].sum()))
    return aval_corr
df_novo['avaliacao_criticos_novo'] = gb.apply(aval_critic)[0]

def aval_usu(c):
    var=[i[1].loc[:,['avaliacao-usuarios', 'numero-usuarios']] for i in gb['avaliacao-usuarios', 'numero-usuarios']]
    aval_corr = []
    for i in var:
        peso_aval_soma=0
        for tab in range(len(i)):
            aux = i.iloc[tab]
            peso_aval_soma += aux[0]*aux[1]
        aval_corr.append((peso_aval_soma)/(i.iloc[:,1].sum()))
    return aval_corr
df_novo['avaliacao_usuarios_novo'] = gb.apply(aval_usu)[0]

/home/gabic9814/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.
/home/gabic9814/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  app.launch_new_instance()


In [139]:
df_novo

,nome,genero,editora,vendas,X360,Wii,PSP,PS3,PS4,3DS,PSV,XOne,WiiU,avaliacao_criticos_novo,numero_criticos_novo,avaliacao_usuarios_novo,numero_usuarios_novo,fabricante
0,2010 FIFA World Cup South Africa,2,Electronic Arts,2.80,1,1,1,1,0,0,0,0,0,NaN,147.0,NaN,107.0,Electronic Arts
1,50 Cent: Blood on the Sand,3,THQ,0.37,1,0,0,1,0,0,0,0,0,NaN,91.0,NaN,99.0,Swordfish Studios
2,ATV Offroad Fury Pro,9,Sony Computer Entertainment,0.43,0,0,1,0,0,0,0,0,0,NaN,32.0,NaN,0.0,Climax Group
3,ATV Quad Kings,9,Zoo Digital Publishing,0.22,0,1,0,0,0,0,0,0,0,NaN,4.0,NaN,0.0,Beyond Reality
4,Ace Combat X: Skies of Deception,11,Namco Bandai Games,0.97,0,0,1,0,0,0,0,0,0,NaN,26.0,NaN,27.0,Access Games
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,Zoo Hospital,11,Majesco Entertainment,0.16,0,1,0,0,0,0,0,0,0,NaN,4.0,NaN,0.0,Torus Games
835,de Blob,10,THQ,0.94,0,1,0,0,0,0,0,0,0,NaN,48.0,NaN,65.0,Blue Tongue Entertainment
836,de Blob 2,10,THQ,0.43,1,1,0,1,0,0,0,0,0,NaN,124.0,NaN,69.0,Blue Tongue Entertainment
837,pro evolution soccer 2011,2,Konami Digital Entertainment,3.98,1,1,1,1,0,0,0,0,0,NaN,103.0,NaN,108.0,Konami
